# 1. Procesado de datos

In [15]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train.csv')
pd.set_option('max_colwidth', 1)
df

,label,text,id
0,0,"Hi Roy hope you are ok, Trans people are not gay. also the only thing thats being rammed down your throat is information about inequality, prejudice and the lack or recognition that 15,000 american trans soldiers have made a pledge that if needs be, they will sacrifice their life for the sake of others. Anyone who is able to join military services to support their country shouldn't be persecuted with such venom. Maybe more appreciation, more tolerance and acceptance and then less wars.",95e98db99c2
1,0,But fuckin' hell what even is biology,a87b8708e63
2,0,Whose the nice looking clergyman?,5b3cb03803f
3,1,"AIDS ARE IN YOUR WAY, SIN HAS CONSEQUENCES AND VERY BAD.",5b0bad2347e
4,0,to learn,332048bd188
...,...,...,...
8143,1,Yeah...if we still alive at the time of the next election. Anything can happen and fairly soon. UK is not safe either.,9141edf7189
8144,0,There is something fundamentally wrong with this,fde30735725
8145,1,"This always confused me. If homosexuality is ""no big deal"", which it isn't, why be proud of it? It's normal, so it isn't an accomplishment. So why celebrate it? I could care less, but I just don't get it.",3d3456b6659
8146,1,disgusting,eace075fe4d


In [3]:
df.isnull().sum()

label    0
text     0
id       0
dtype: int64

In [6]:
df.shape

(8148, 3)

In [7]:
df['label'].value_counts()

label
0    5172
1    2976
Name: count, dtype: int64

In [8]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [ ]:
en_stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text

    return cleanText

df['text_cleaned'] = df['text'].apply(remove_stop_words)

In [ ]:
df['text_cleaned']